In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 9.5 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=10cf3971926b649f5b0df005bce90a800c659f2fa5767345ac0069cf83e2e2b3
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [7]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions - data conversion

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        if 'T' in date_string:
            return date_string.split('T')[0]

        if re.match(r'\d{2}\.\d{2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
            changed_date = datetime.fromtimestamp(mktime(result))
            return format(changed_date.date())

        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        return "no date"


#%% functions - link extraction

def get_all_article_links():
    """
    Pobiera wszystkie linki do artykułów z qfant.pl
    Struktura: <article> z linkiem w <a href="/article-name.html">
    """
    base_url = "https://www.qfant.pl"

    print("="*60)
    print("KROK 1: Pobieranie linków do artykułów")
    print("="*60 + "\n")

    all_links = set()

    # Kategorie do scrapowania (z menu głównego)
    categories = {
        'aktualności': '/cons/aktualnosci.html',
        'artykuły': '/cons/artykuly.html',
        'nowości wydawnicze': '/cons/nowosci-wydawnicze.html',
        'patronaty': '/cons/patronaty.html',
        'konkursy': '/cons/konkursy.html',
        'wyniki': '/cons/wyniki.html',
        'wywiady': '/cons/wywiady.html',
        'wydarzenia': '/cons/wydarzenia.html',
        'recenzje': '/cons/recenzja.html',
    }

    for cat_name, cat_url in categories.items():
        print(f"Pobieranie linków z kategorii: {cat_name}...")

        # qfant.pl ma paginację - próbujemy pobrać kilka pierwszych stron
        for page in range(1, 6):  # pierwsze 5 stron każdej kategorii
            try:
                if page == 1:
                    url = base_url + cat_url
                else:
                    # Paginacja to /page/2.html, /page/3.html itd.
                    url = f"{base_url}{cat_url.replace('.html', '')}/page/{page}.html"

                r = requests.get(url, timeout=10)
                r.encoding = 'utf-8'

                if r.status_code == 200:
                    soup = BeautifulSoup(r.text, 'lxml')

                    # Szukamy artykułów - <article> z linkami w tytule
                    # <h5><a href="/copernicon-2016.html">Copernicon 2016</a></h5>
                    articles = soup.find_all('article', class_='post-item')

                    page_links = 0
                    for article in articles:
                        # Znajdź link w tytule artykułu
                        title_div = article.find('div', class_='post-title')
                        if title_div:
                            link = title_div.find('a')
                            if link and link.get('href'):
                                href = link['href']

                                # Upewniamy się, że to pełny URL
                                if href.startswith('http'):
                                    full_url = href
                                elif href.startswith('/'):
                                    full_url = base_url + href
                                else:
                                    full_url = base_url + '/' + href

                                # Dodajemy tylko linki do artykułów (kończy się na .html)
                                if '.html' in full_url and '/page/' not in full_url:
                                    all_links.add(full_url)
                                    page_links += 1

                    print(f"  Strona {page}: {page_links} linków")

                    if page_links == 0:
                        break  # Jeśli nie ma więcej artykułów, przerywamy paginację

                    time.sleep(0.5)  # Aby nie przeciążać serwera

                else:
                    print(f"  ✗ Status {r.status_code} dla strony {page}")
                    break

            except Exception as e:
                print(f"  ✗ Błąd na stronie {page}: {e}")
                break

        print(f"  ✓ Zebrano linki z kategorii {cat_name}\n")

    # Konwertujemy set na listę
    all_links = list(all_links)

    print(f"{'='*60}")
    print(f"Łącznie znaleziono: {len(all_links)} unikalnych artykułów")
    print(f"{'='*60}\n")

    return all_links


#%% functions - scraping

def dictionary_of_article(article_link):
    """
    Pobiera szczegóły artykułu z qfant.pl
    """
    try:
        r = requests.get(article_link, timeout=15)
        r.encoding = 'utf-8'

        if r.status_code != 200:
            errors.append(article_link)
            return

        html_text = r.text

        if 'error 404' in html_text.lower() or 'page not found' in html_text.lower():
            errors.append(article_link)
            return

        soup = BeautifulSoup(html_text, 'lxml')

        # Data publikacji
        try:
            # W HTML: <div class="post-date">17 października 2016</div>
            date_div = soup.find('div', class_='post-date')
            if date_div:
                date_text = date_div.get_text(strip=True)
                date_of_publication = date_change_format(date_text)
            else:
                date_of_publication = "no date"
        except:
            date_of_publication = "no date"

        # Tytuł
        try:
            # W HTML: <div class="title-caption-large"><h3>Copernicon 2016</h3></div>
            title_div = soup.find('div', class_='title-caption-large')
            if title_div:
                h3 = title_div.find('h3')
                if h3:
                    title = h3.get_text(strip=True)
                else:
                    title = "no title"
            else:
                title = "no title"
        except:
            title = "no title"

        # Autor
        try:
            # Autor jest zwykle na końcu artykułu: "Autor: Łukasz Szatkowski"
            author = "no author"
            post_entry = soup.find('section', class_='post-entry')
            if post_entry:
                # Szukamy "Autor:" lub "Recenzował:" lub "Recenzja:"
                text = post_entry.get_text()

                # Wzorce dla autora
                patterns = [
                    r'Autor:\s*([^\n]+)',
                    r'Recenzował[aą]?:\s*([^\n]+)',
                    r'Recenzja:\s*([^\n]+)',
                ]

                for pattern in patterns:
                    match = re.search(pattern, text, re.IGNORECASE)
                    if match:
                        author = match.group(1).strip()
                        # Usuń ewentualne tagi HTML
                        author = re.sub(r'<[^>]+>', '', author)
                        break
        except:
            author = "no author"

        # Treść artykułu
        try:
            # Główna treść jest w <section class="post-entry">
            post_entry = soup.find('section', class_='post-entry')

            if post_entry:
                # Usuń niepotrzebne elementy
                for element in post_entry.find_all(['script', 'style', 'iframe', 'ins',
                                                     'div']):
                    # Zostawiamy paragrafy, ale usuwamy divy i inne
                    if element.name == 'div' and element.get('class'):
                        # Usuń divy z datą, tytułem, oceną
                        if any(cls in ['post-date', 'post-title', 'star', 'clear']
                               for cls in element.get('class', [])):
                            element.decompose()

                # Wyciągamy tekst z paragrafów
                paragraphs = post_entry.find_all('p')
                text_parts = []
                for p in paragraphs:
                    p_text = p.get_text(strip=True)
                    # Pomijamy paragrafy z autorem (będzie osobno)
                    if p_text and not any(skip in p_text for skip in
                                        ['Autor:', 'Recenzował', 'Recenzja:']):
                        text_parts.append(p_text)

                text = ' '.join(text_parts)
                text = text.replace('\n', ' ').replace('\xa0', ' ')
                text = re.sub(r'\s+', ' ', text).strip()
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategoria - z tagów "platform-teaser"
        try:
            # <a style="background-color: #F18F19" class="game_console" href="/cons_artykuly.html">Artykuły</a>
            categories = []
            platform_teaser = soup.find('div', class_='platform-teaser')
            if platform_teaser:
                links = platform_teaser.find_all('a', class_='game_console')
                for link in links:
                    cat_name = link.get_text(strip=True)
                    if cat_name:
                        categories.append(cat_name)

            if categories:
                category = ', '.join(categories)
            else:
                category = "no category"
        except:
            category = "no category"

        # Tagi
        tags_str = None

        # Linki zewnętrzne
        try:
            post_entry = soup.find('section', class_='post-entry')
            if post_entry:
                links = [a['href'] for a in post_entry.find_all('a', href=True)]
                external_links = [link for link in links
                                if not re.search(r'qfant\.pl', link)
                                and link.startswith('http')]
                external_links = ' | '.join(external_links) if external_links else None
            else:
                external_links = None
        except:
            external_links = None

        # Zdjęcia
        try:
            images = []

            # Główne zdjęcie artykułu - w <section class="post-thumb">
            post_thumb = soup.find('section', class_='post-thumb')
            if post_thumb:
                main_img = post_thumb.find('img')
                if main_img and main_img.get('src'):
                    img_url = main_img['src']
                    if not img_url.startswith('http'):
                        img_url = 'https://www.qfant.pl' + img_url
                    if img_url not in images:
                        images.append(img_url)

            # Wszystkie zdjęcia z treści artykułu
            post_entry = soup.find('section', class_='post-entry')
            if post_entry:
                all_imgs = post_entry.find_all('img')
                for img in all_imgs:
                    img_url = img.get('src') or img.get('data-lazy-src')
                    if img_url:
                        # Pomijamy małe ikony
                        if not any(x in img_url.lower() for x in ['icon', 'logo', 'star-']):
                            if not img_url.startswith('http'):
                                img_url = 'https://www.qfant.pl' + img_url
                            if img_url not in images:
                                images.append(img_url)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except:
            has_images = False
            photos_links = None

        # Filmy
        try:
            has_videos = False
            post_entry = soup.find('section', class_='post-entry')
            if post_entry:
                iframes = post_entry.find_all('iframe', src=True)
                has_videos = len(iframes) > 0
        except:
            has_videos = False

        result = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Tagi": tags_str,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(result)

    except Exception as e:
        errors.append(article_link)


#%% main execution

if __name__ == "__main__":
    print("\n" + "="*60)
    print("SCRAPER QFANT.PL")
    print("Dwumiesięcznik fantastyczno-kryminalny")
    print("="*60 + "\n")

    # KROK 1: Pobierz linki do artykułów
    article_links = get_all_article_links()

    if not article_links:
        print("Nie znaleziono artykułów!")
        exit(1)

    # KROK 2: Scrapuj artykuły
    all_results = []
    errors = []

    print("="*60)
    print("KROK 2: Scraping artykułów")
    print("="*60 + "\n")

    max_workers = 10
    print(f"Używam {max_workers} równoległych wątków\n")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # KROK 3: Zapisz wyniki
    timestamp = datetime.today().date()

    print(f"\n{'='*60}")
    print("KROK 3: Zapisywanie wyników")
    print("="*60)

    # JSON
    json_file = f'qfant_{timestamp}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)
    print(f"  ✓ {json_file}")

    # Excel
    excel_file = f"qfant_{timestamp}.xlsx"
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(excel_file,
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)
    print(f"  ✓ {excel_file}")

    # RAPORT KOŃCOWY
    print(f"\n{'='*60}")
    print("RAPORT KOŃCOWY")
    print("="*60)
    print(f"Pobranych artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")

    if errors and len(errors) <= 10:
        print(f"\nLinki z błędami:")
        for error_link in errors:
            print(f"  - {error_link}")
    elif errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        print(f"  ... i {len(errors) - 10} więcej")

    print(f"\n{'='*60}")
    print("GOTOWE!")
    print("="*60 + "\n")


SCRAPER QFANT.PL
Dwumiesięcznik fantastyczno-kryminalny

KROK 1: Pobieranie linków do artykułów

Pobieranie linków z kategorii: aktualności...
  Strona 1: 15 linków
  Strona 2: 15 linków
  Strona 3: 15 linków
  Strona 4: 14 linków
  Strona 5: 14 linków
  ✓ Zebrano linki z kategorii aktualności

Pobieranie linków z kategorii: artykuły...
  Strona 1: 15 linków
  Strona 2: 15 linków
  Strona 3: 15 linków
  Strona 4: 15 linków
  Strona 5: 7 linków
  ✓ Zebrano linki z kategorii artykuły

Pobieranie linków z kategorii: nowości wydawnicze...
  Strona 1: 15 linków
  Strona 2: 12 linków
  Strona 3: 15 linków
  Strona 4: 15 linków
  Strona 5: 13 linków
  ✓ Zebrano linki z kategorii nowości wydawnicze

Pobieranie linków z kategorii: patronaty...
  Strona 1: 15 linków
  Strona 2: 12 linków
  Strona 3: 14 linków
  Strona 4: 14 linków
  Strona 5: 14 linków
  ✓ Zebrano linki z kategorii patronaty

Pobieranie linków z kategorii: konkursy...
  Strona 1: 9 linków
  Strona 2: 13 linków
  Strona 3: 15 li

100%|██████████| 479/479 [00:48<00:00,  9.87it/s]
/tmp/ipython-input-1050694422.py:416: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Posts', index=False)



KROK 3: Zapisywanie wyników
  ✓ qfant_2026-01-13.json
  ✓ qfant_2026-01-13.xlsx

RAPORT KOŃCOWY
Pobranych artykułów: 411
Błędów: 68

Linki z błędami (pierwsze 10):
  - https://www.qfant.pl/qfantydzien-z-prawem-do-zemsty-01-wyniki.html
  - https://www.qfant.pl/qfantydzien-z-krucjata-05.html
  - https://www.qfant.pl/karciany-qfantydzien-01.html
  - https://www.qfant.pl/13746-2.html
  - https://www.qfant.pl/grzmiacy-qfantydzien-04-wyniki.html
  - https://www.qfant.pl/kwantowy-zlodziej-hannu-rajaniemi.html
  - https://www.qfant.pl/qfantydzien-ze-sciana-burz-01-03-05-i-07-wyniki.html
  - https://www.qfant.pl/qfantydzien-z-andromeda-05-wyniki.html
  - https://www.qfant.pl/warcraft-narodziny-hordy-premiera.html
  - https://www.qfant.pl/qfantydzien-z-andromeda-03-2.html
  ... i 58 więcej

GOTOWE!



In [8]:
df.head()

,Link,Data publikacji,Tytuł artykułu,Tekst artykułu,Autor,Kategoria,Tagi,Linki zewnętrzne,Zdjęcia/Grafika,Filmy,Linki do zdjęć
0,https://www.qfant.pl/review/tom-rob-smith-farm...,2015-11-11,Tom Rob Smith „Farma”,Tom Rob Smith to autor rewelacyjnego „Systemu”...,Damian Drabik,Recenzja,None,None,True,False,https://www.qfant.pl/wp-content/uploads/2015/1...
1,https://www.qfant.pl/review/wilcza-godzina-and...,2017-07-25,"„Wilcza godzina”, Andrius Tapinas",Nieczęsto na polskim rynku mamy do czynienia z...,Andrus Tapinas,Recenzja,None,None,True,False,https://www.qfant.pl/wp-content/uploads/2017/0...
2,https://www.qfant.pl/j-d-bujak-ogniskowa.html,2016-09-22,"J. D. Bujak, „Ogniskowa”","Emilia jest z pozoru zwyczajną, skromną studen...",no author,"Nowości wydawnicze, Patronaty",None,None,True,False,https://www.qfant.pl/wp-content/uploads/2016/0...
3,https://www.qfant.pl/review/vittorio-sgarbi-ob...,2016-01-03,Vittorio Sgarbi „Oblicza kobiet w sztuce”,"Nie ulega wątpliwości, że najdoskonalszą chwil...",Damian Drabik,Recenzja,None,None,True,False,https://www.qfant.pl/wp-content/uploads/2016/0...
4,https://www.qfant.pl/qfant-14.html,2011-03-01,QFANT #14,ŚCIĄGNIJ PDF Od dziś możecie się zapoznać z na...,no author,"Aktualności, Wydarzenia",None,None,True,False,https://www.qfant.pl/wp-content/uploads/2013/0...
